In [ ]:
import torch
from torch import Tensor
from matplotlib import pyplot as plt
from baukit import PlotWidget, show

class Sign(torch.nn.Module):
    def forward(self, x):
        if isinstance(x, dict):
            return {k: v.sign() for k, v in x.items()}
        return x.sign()

x = torch.linspace(-2, 2, 101)
plt.title('Sign (step) nonlinearity')
plt.plot(x, Sign()(x))
plt.xlabel('input')
plt.ylabel('output')

In [ ]:
class McCulloughPittsNeuron(torch.nn.Module):
    '''
    A McCoullough-Pitts Neuron.  It computes a weighted sum of any number of inputs,
    then it thresholds the output through a nonlinear activation step function.
    It pulls named inputs from an input dictionary and puts output into the
    dictionary.  That allows networks to be created by sequencing neurons and
    connecting them by using dictionary names.

    Examples:

        net = McCulloughPittsNeuron(
                weight_a = 0.5,
                weight_b = -0.3,
                weight_c = 2.0,
                bias     = 1.0)
        print(net(dict(
                a=Tensor([1.0]),
                b=Tensor([-1.0]),
                c=Tensor([-1.0])))['out'])

    The above creates a single neuron with three inputs a, b, and c plus some bias.
    It is invoked by providing a dictionary of all the inputs as tensors.

        net = torch.nn.Sequential(
            McCulloughPittsNeuron(weight_a=-1.0, weight_b=1.0, output_name='d'),
            McCulloughPittsNeuron(weight_b=1.0, weight_d=1.0, bias=1.0),
        )
        print(net(dict(a=Tensor([1.0]), b=Tensor([-1.0])))['out'])

    The above creates and runs a network of two neurons in this configuration:
    ```
             a -----> +----------+
                      | Neuron 0 | ---> d --+
             b ---+-> +----------+          +--> +----------+
                  |                              | Neuron 1 | ---> out
                  +----------------------------> +----------+
    ```
    As the sequence is run, the dictionary grows; after the first neuron is run,
    the dictionary contains a, b, and d.  After the second neuron is ru , the
    final dictionary contains a, b, d, and out.
    '''
    def __init__(self, bias=0.0, activation=Sign, output_name='out', **kwargs):
        '''
        Construct a neuron by specifying any number of input weights in the arguments:
        
            weight_a:    The weight for the 'a' input.
                         Each `weight_x` in the constructor adds an input named 'x'.
            bias:        The constant bias to add to the weighted sum.
            output_name: The output name, defaults to 'out'.
            activation:  The nonlinearity to use; defaults to the "Sign" step function.
        '''
        super().__init__()
        
        # We use the pytorch Linear module with a one-dimenaional output
        self.summation = torch.nn.Linear(len(kwargs), 1)
        self.activation = None if activation is None else activation()
        self.output_name = output_name
        self.input_names = []
        with torch.no_grad():
            self.summation.bias[...] = bias
            for k, v in kwargs.items():
                assert k.startswith('weight_'), f'Bad argument {k}'
                self.summation.weight[0, len(self.input_names)] = v
                self.input_names.append(k[7:])

    def forward(self, inputs):
        '''
        The inputs should be a dictionary containing the expected input keys.
        The results are computed.  Then the return value will be a copy of the
        input dictionary, with the additional output tensor added.
        '''
        state = inputs.copy()
        assert self.output_name not in state, f'Multiple {self.output_name}\'s conflict'
        x = torch.stack([inputs[v] for v in self.input_names], dim=1)
        x = self.summation(x)[:,0]
        if self.activation is not None:
            x = self.activation(x)
        state[self.output_name] = x
        return state
    
    def extra_repr(self):
        return f'input_names={self.input_names}, output_name=\'{self.output_name}\''

def visualize_logic(nets, arg1='a', arg2='b'):
    '''
    Pass any number of McCoullough-Pitts neurons or neural networks with two
    inputs named 'a' and 'b', and it will visualize all of their logic, using
    white squares to indicate +1, black squares to indicate -1, and orange
    squares to indicate intermediate values.
    '''
    grid = torch.Tensor([[
        [-1.0, 1.0],
        [-1.0, 1.0],
    ], [
        [ 1.0, 1.0],
        [-1.0,-1.0],
    ]])
    a, b = grid
    def make_viz(n):
        if isinstance(n, list):
            return [make_viz(net) for net in n]
        def make_plot(fig):
            with torch.no_grad():
                out = n({arg1: a.view(-1), arg2: b.view(-1)})['out'].view(a.shape)
            [ax] = fig.axes
            ax.imshow(out, cmap='hot', extent=[-2,2,-2,2], vmin=-1, vmax=1)
            ax.invert_yaxis()
            ax.xaxis.tick_top()
            ax.tick_params(length=0)
            ax.set_xticks([-1, 1], [f'{arg1}=-1', f'{arg1}=1'])
            ax.set_yticks([-1, 1], [f'{arg2}=-1', f'{arg2}=1'])
        return PlotWidget(make_plot, figsize=(1.1,1.1), dpi=100, bbox_inches='tight')
    show(show.WRAP, make_viz(nets))




In [ ]:
visualize_logic([
    # First network: just one neuron.
    McCulloughPittsNeuron(weight_a=1.0, weight_b=1.0, bias=0.0),
    
    # Second network: two neurons hooked together.
    torch.nn.Sequential(
        McCulloughPittsNeuron(weight_a=-1.0, weight_b=1.0, bias=0.0, output_name='d'),
        McCulloughPittsNeuron(weight_b=1.0, weight_d=1.0, bias=1.0),
    ),
])


In [ ]:
visualize_logic([
    McCulloughPittsNeuron(weight_a=1.0, weight_b=1.0, bias=1.0),
    McCulloughPittsNeuron(weight_a=1.0, weight_b=-1.0, bias=1.0),
    McCulloughPittsNeuron(weight_a=-1.0, weight_b=-1.0, bias=1.0),
    McCulloughPittsNeuron(weight_a=-1.0, weight_b=1.0, bias=1.0),
    McCulloughPittsNeuron(weight_a=1.0, weight_b=1.0, bias=-1.0),
    McCulloughPittsNeuron(weight_a=1.0, weight_b=-1.0, bias=-1.0),
    McCulloughPittsNeuron(weight_a=-1.0, weight_b=-1.0, bias=-1.0),
    McCulloughPittsNeuron(weight_a=-1.0, weight_b=1.0, bias=-1.0),
    McCulloughPittsNeuron(weight_a=1.0, weight_b=0.0, bias=0.0),
    McCulloughPittsNeuron(weight_a=0.0, weight_b=1.0, bias=0.0),
    torch.nn.Sequential(
        McCulloughPittsNeuron(weight_a=-1.0, weight_b=-1.0, bias=1.0, output_name='c'),
        McCulloughPittsNeuron(weight_a=1.0, weight_b=1.0, bias=1.0, output_name='d'),
        McCulloughPittsNeuron(weight_c=1.0, weight_d=1.0, bias=-1.0),
    ),
    torch.nn.Sequential(
        McCulloughPittsNeuron(weight_a=-1.0, weight_b=-1.0, bias=1.0, output_name='c'),
        McCulloughPittsNeuron(weight_a=1.0, weight_b=1.0, bias=1.0, output_name='d'),
        McCulloughPittsNeuron(weight_c=-1.0, weight_d=-1.0, bias=1.0),
    ),
    McCulloughPittsNeuron(weight_a=-1.0, weight_b=0.0, bias=0.0),
    McCulloughPittsNeuron(weight_a=0.0, weight_b=-1.0, bias=0.0),
])

In [ ]:
net = torch.nn.Sequential(
    McCulloughPittsNeuron(weight_a=-1.0, weight_b=1.0, output_name='c'),
    McCulloughPittsNeuron(weight_b=1.0, weight_c=1.0, bias=1.0),
)
print(net(dict(a=Tensor([1.0]), b=Tensor([-1.0])))['out'])

In [ ]:
net = McCulloughPittsNeuron(weight_a=0.5, weight_b=-0.3, weight_c=2.0, bias=1.0)
print(net(dict(a=Tensor([1.0]), b=Tensor([-1.0]), c=Tensor([-1.0])))['out'])

In [ ]:
net(dict(a=torch.tensor([1.0]).float(), b=torch.tensor([1.0]).float()))

In [ ]:
net

In [ ]:
from matplotlib import pyplot as plt
x = torch.linspace(-10, 10, 101)

plt.plot(x, torch.sigmoid(x))

plt.plot(x, (x > 0))

In [ ]:
(x>0).to(x)